# 60 Run mock pipeline
gold特徴量を読み、run_planごとにMock実行し、DBとParquetに結果を書き込む。

In [1]:
import pandas as pd
from fcast.lake.paths import table_path
from fcast.lake.io import read_parquet
from fcast.meta.db import session_scope, init_db
from fcast.meta.orm import RunsPlan, ModelsRegistry, ModelsSeriesSelector
from fcast.pipeline.runner import mock_run_plan
init_db()


In [2]:
gold_path = table_path("gold","gold_loto6_features")
df = read_parquet(gold_path)
df.head()


,unique_id,ds,y,hist_y_diff1,hist_y_cumsum,hist_y_roll_sum_3,hist_y_roll_mean_3,hist_y_roll_sum_7,hist_y_roll_mean_7
0,N1,2020-01-05,22,NaN,22,NaN,NaN,NaN,NaN
1,N1,2020-01-12,24,2.0,46,NaN,NaN,NaN,NaN
2,N1,2020-01-19,34,10.0,80,80.0,26.666667,NaN,NaN
3,N1,2020-01-26,42,8.0,122,100.0,33.333333,NaN,NaN
4,N1,2020-02-02,3,-39.0,125,79.0,26.333333,NaN,NaN


In [3]:
# For simplicity, choose a single run_plan (first) and run.
with session_scope() as s:
    plan = s.query(RunsPlan).first()
    print("run_id:", plan.run_id, "model_id:", plan.model_id)
run_id = plan.run_id


AttributeError: 'NoneType' object has no attribute 'run_id'

In [ ]:
mock_run_plan(run_id=run_id, df=df, preds_h=5)
print("done")


In [ ]:
# Inspect DB tables: validations, results, metrics, artifacts
import pandas as pd
from fcast.meta.db import session_scope
from fcast.meta.orm import RunsValidation, RunsResult, RunsMetric, RunsArtifact, RunsStep

with session_scope() as s:
    steps = s.query(RunsStep).filter_by(run_id=run_id).order_by(RunsStep.ord).all()
    vals = s.query(RunsValidation).join(RunsStep, RunsValidation.run_step_id==RunsStep.run_step_id).filter(RunsStep.run_id==run_id).all()
    res = s.query(RunsResult).join(RunsStep, RunsResult.run_step_id==RunsStep.run_step_id).filter(RunsStep.run_id==run_id).all()
    met = s.query(RunsMetric).join(RunsStep, RunsMetric.run_step_id==RunsStep.run_step_id).filter(RunsStep.run_id==run_id).all()
    art = s.query(RunsArtifact).filter_by(run_id=run_id).all()

pd.DataFrame([{"step":st.step_name,"ord":st.ord} for st in steps]), \
pd.DataFrame([{"ok":v.ok,"violations":len(v.violations)} for v in vals]), \
pd.DataFrame([{"status":r.status} for r in res]), \
pd.DataFrame([{"metric":m.metric_name,"value":m.metric_value} for m in met]), \
pd.DataFrame([{"type":a.artifact_type,"uri":a.storage_uri} for a in art])


In [ ]:
# Read prediction parquet
from fcast.meta.db import session_scope
from fcast.meta.orm import RunsArtifact
from fcast.lake.io import read_parquet
with session_scope() as s:
    art = s.query(RunsArtifact).filter_by(run_id=run_id, artifact_type="predictions").first()
pred = read_parquet(__import__("pathlib").Path(art.storage_uri))
pred.head()
